## Convert dynamic computational graph to ONNX

#### Prepare environment

In [ ]:
!pip install paddlepaddle paddle2onnx # required for paddle.onnx.export, paddlepaddle>=2.0.0, paddle2onnx>=0.3.2
!pip install onnx onnxruntime # optional for check and run ONNX model, onnx>=1.7.0, onnxruntime>=1.5.2

### 1、Build dynamic computational Network 

In [1]:
import paddle
from paddle import nn
from paddle.static import InputSpec
import paddle2onnx as p2o

class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear = nn.Linear(784, 10)

    def forward(self, x):
        return self._linear(x)

### 2、Export model to ONNX

In [2]:
# export to ONNX 
layer = LinearNet()
save_path = 'onnx.save/linear_net'
x_spec = InputSpec([None, 784], 'float32', 'x')
layer.eval()
paddle.onnx.export(layer, save_path, input_spec=[x_spec])
# p2o.dygraph2onnx(layer, save_path+'.onnx', input_spec=[x_spec])

2020-11-18 07:00:39 [INFO]	ONNX model saved in onnx.save/linear_net.onnx


### 3、Verify the correctness of ONNX model

In [3]:
# check by ONNX
import onnx

onnx_file = save_path + '.onnx'
onnx_model = onnx.load(onnx_file)
onnx.checker.check_model(onnx_model)
print('The model is checked!')

The model is checked!


### 4、Inference ONNX model by ONNXRuntime

In [4]:
import numpy as np
import onnxruntime

x = np.random.random((2, 784)).astype('float32')

# predict by ONNX Runtime
ort_sess = onnxruntime.InferenceSession(onnx_file)
ort_inputs = {ort_sess.get_inputs()[0].name: x}
ort_outs = ort_sess.run(None, ort_inputs)

print("Exported model has been predict by ONNXRuntime!")

Exported model has been predict by ONNXRuntime!


### 5、compare ONNX Runtime and Paddle results

In [5]:
# predict by Paddle
paddle_outs = layer(x)

# compare ONNX Runtime and Paddle results
np.testing.assert_allclose(
    ort_outs[0], paddle_outs.numpy(), rtol=1.0, atol=1e-05)

print("The difference of result between ONNXRuntime and Paddle looks good!")

The difference of result between ONNXRuntime and Paddle looks good!
